In [39]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [40]:
# Adapter Layer

def csv_to_df(bucket,key,decoding = 'utf-8',sep = ','):
    csv_obj = bucket.Object(key=key).get().get("Body").read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data,delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer,index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key = key)
    return True

def list_files_in_prefix(bucket,prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix = prefix)]
    return files



In [41]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket,date)]
    df = pd.concat([csv_to_df(bucket,obj) for obj in files], ignore_index = True)
    return df
    
def transform_report1(df, columns,arg_date):
    df = df.loc[:,columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by = "Time").groupby(['ISIN','Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by = "Time").groupby(['ISIN','Date'])['EndPrice'].transform('last')
    df = df.groupby(['ISIN','Date'], as_index = False).agg(opening_price_eur=('opening_price','min'),closing_price_eur = ('closing_price','min'),minimum_price_eur=('MinPrice','min'),maximum_price_eur=('MaxPrice','max'),daily_traded_volume=('TradedVolume','sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns = ['prev_closing_price'],inplace = True)
    df = df.round(decimals = 2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket,df,trg_key,trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket,date_list)
    df = transform_report1(df,columns,arg_date)
    load(trg_bucket,df, trg_key, trg_format)
    return True


In [42]:
# Application Layer - not core

def return_date_list(bucket,arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    return_date_list = [(min_date + timedelta(days = x)).strftime(src_format) for x in range(0,(today-min_date).days + 1)]
    return return_date_list

In [43]:
# Main Function Entrypoint

def main():
    # Parameters/configurations
    # Later read config
    arg_date = "2021-08-29"
    src_format = "%Y-%m-%d"
    src_bucket = "deutsche-boerse-xetra-pds"
    trg_bucket = 'xetra-daily'
    columns = ['ISIN','Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report'
    trg_format = '.parquet'

    # Initialize
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)

    # Run application
    date_list = return_date_list(bucket_src,arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, date_list, columns, arg_date, trg_key, trg_format)


In [44]:
# Run

main()

## Reading the uploaded file

In [48]:
s3 = boto3.resource('s3')
trg_bucket = 'xetra-daily'
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report20210829_080542.parquet
xetra_daily_report20210829_094217.parquet
xetra_daily_report20210831_134004.parquet
xetra_daily_report20210831_135925.parquet


In [49]:
prq_obj = bucket_trg.Object(key="xetra_daily_report20210831_135925.parquet").get().get("Body").read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [50]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-08-30,8.84,8.75,8.75,8.84,240,NaN
1,AT0000606306,2021-08-30,20.24,20.38,20.18,20.38,837,NaN
2,AT0000644505,2021-08-30,112.80,112.60,112.60,112.80,1,NaN
3,AT0000652011,2021-08-30,33.53,33.69,33.43,33.69,303,NaN
4,AT0000652250,2021-08-30,20.40,20.70,20.40,20.70,659,NaN
...,...,...,...,...,...,...,...,...
3042,XS2265369731,2021-08-30,8.59,8.51,8.51,8.60,300,NaN
3043,XS2265370234,2021-08-30,20.50,20.89,20.50,20.89,0,NaN
3044,XS2284324667,2021-08-30,30.07,29.87,29.77,30.07,3856,NaN
3045,XS2314659447,2021-08-30,8.04,8.15,8.04,8.15,0,NaN
